In [11]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
import torch.nn as nn
import torchvision
import seaborn as sn

import os
import time
import copy

from tqdm import tqdm

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd


from sklearn import metrics
from sklearn.metrics import (f1_score, balanced_accuracy_score,
   classification_report, confusion_matrix, roc_curve, auc)

import torchvision.models as models

if torch.cuda.is_available():
    print('CUDA is available. Working on GPU')
    DEVICE = torch.device('cuda')
else:
    print('CUDA is not available. Working on CPU')
    DEVICE = torch.device('cpu')

CUDA is not available. Working on CPU


In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
path_dir = '/content/gdrive/My Drive/FairDerm-Colab/'

In [14]:
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2):
        super(EnsembleModel, self).__init__()

        # Set the two models as the attributes
        self.model1 = model1
        self.model2 = model2
        # Define the new FC layer for the ensemble model
        self.classifier = nn.Linear(4, 2)

    def forward(self, x):
        output1 = self.model1(x)
        output2 = self.model2(x)


        output_ = torch.cat((output1, output2), dim=1)
        # Pass combined output through new FC layer
        output = self.classifier(output_)
        return output

In [15]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = os.path.abspath(self.imgs[index][0])
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [16]:
image_dir = path_dir + "/dataset/combined/"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])


# pre-processnig (modify it to just one image)
input_size = 299
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(degrees=20),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomResizedCrop(input_size),
        transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1),
        transforms.GaussianBlur(kernel_size=(5,9),sigma=(0.1,5)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'temp': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# Create training and validation datasets
image_datasets_combined = {x: datasets.ImageFolder(os.path.join(image_dir, x), data_transforms[x]) for x in ['train','temp','val']}
# Create training and validation dataloaders
dataloaders_dict_combined = {x: torch.utils.data.DataLoader(image_datasets_combined[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train','temp','val']}

In [17]:
image_dir = path_dir + "/dataset/12/"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])


input_size = 299
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(degrees=20),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomResizedCrop(input_size),
        transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1),
        transforms.GaussianBlur(kernel_size=(5,9),sigma=(0.1,5)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'temp': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# Create training and validation datasets
image_datasets_12 = {x: datasets.ImageFolder(os.path.join(image_dir, x), data_transforms[x]) for x in ['train','temp', 'val']}
# Create training and validation dataloaders
dataloaders_dict_12 = {x: torch.utils.data.DataLoader(image_datasets_12[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train','temp', 'val']}

In [18]:
image_dir =  path_dir + "/dataset/56/"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])



input_size = 299
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(degrees=20),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomResizedCrop(input_size),
        transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1),
        transforms.GaussianBlur(kernel_size=(5,9),sigma=(0.1,5)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'temp': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# Create training and validation datasets
image_datasets_56 = {x: datasets.ImageFolder(os.path.join(image_dir, x), data_transforms[x]) for x in ['train','temp', 'val']}
# Create training and validation dataloaders
dataloaders_dict_56 = {x: torch.utils.data.DataLoader(image_datasets_56[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'temp', 'val']}

In [19]:
baseline = torch.load(path_dir + '/new_models/deepderm_base.pth')
model_full = torch.load(path_dir + '/new_models/combined_temp_FT.pth')
model12 = torch.load(path_dir + '/new_models/12_temp_FT.pth')
model56 = torch.load(path_dir + '/new_models/56_temp_FT.pth')

# Our models (drop down list)
ensemble_vanilla = torch.load(path_dir + '/new_models/ensemble_vanilla_weight_june_12.pth', map_location=DEVICE)
ensemble_label = torch.load(path_dir + '/new_models/ensemble_label_weight_june_12.pth', map_location=DEVICE)
ensemble_sens = torch.load(path_dir + '/new_models/ensemble_sens_weight_june_12.pth', map_location=DEVICE)
ensemble_joint = torch.load(path_dir +'/new_models/ensemble_inverse_joint_weight_june_13.pth', map_location=DEVICE)
ensemble_kamiran = torch.load(path_dir +'/new_models/ensemble_kamiran.pth', map_location=DEVICE)



In [20]:
# Modify
def eval_model(model, testloader, use_gpu=False, show_plot=False):

    DEVICE = torch.device('cpu')

    # prepare model for evaluation
    model.to(DEVICE).eval()

    y_prob = []
    y_pred = []
    y_true = []
    with torch.no_grad():
        for i, batch in tqdm(enumerate(testloader)):

            inputs, labels = batch
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs) # prob (0-1) (####)

            pred = outputs.argmax(dim=1) # class (####)
            pred = pred.data.cpu().numpy() #
            y_pred.extend(pred)

            prob = outputs[:, 1]
            prob = prob.data.cpu().numpy()
            y_prob.extend(prob)

            labels = labels.data.cpu().numpy()
            y_true.extend(labels) # Save Truth


    accuracy = metrics.accuracy_score(y_true, y_pred)
    balanced_accuracy = metrics.balanced_accuracy_score(y_true, y_pred)
    auc = metrics.roc_auc_score(y_true, y_prob)
    f1 = metrics.f1_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)


    # True positive
    tp = np.sum(y_true * y_pred)
    # False positive
    fp = np.sum((y_true == 0) * y_pred)
    # True negative
    tn = np.sum((y_true==0) * (y_pred==0))
    # False negative
    fn = np.sum(y_true * (y_pred==0))

    # True positive rate (sensitivity or recall)
    tpr = tp / (tp + fn)
    # False positive rate (fall-out)
    fpr = fp / (fp + tn)
    # True Negative Rate
    tnr = 1 - fpr
    # False Negative Rate
    fnr = 1 - tpr



    eval_results = {'accuracy': accuracy,
                    'balanced_accuracy': balanced_accuracy,
                    'auc': auc,
                    'f1' : f1,
                    'precision': precision,
                    'tpr': recall,
                    'fpr': fpr,
                    'tnr': tnr,
                    'fnr': fnr

    }



    return eval_results

In [22]:
pd.DataFrame.from_dict(eval_model(baseline, dataloaders_dict_combined['val']), orient='index')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
12it [02:23, 11.99s/it]


,0
accuracy,0.786704
balanced_accuracy,0.776098
auc,0.859621
f1,0.821346
precision,0.804545
tpr,0.838863
fpr,0.286667
tnr,0.713333
fnr,0.161137


In [ ]:
pd.DataFrame.from_dict(eval_model(baseline, dataloaders_dict_12['val']), orient='index')

9it [01:58, 13.16s/it]


,0
accuracy,0.799257
balanced_accuracy,0.786597
auc,0.872261
f1,0.837349
precision,0.832335
tpr,0.842424
fpr,0.269231
tnr,0.730769
fnr,0.157576


In [ ]:
pd.DataFrame.from_dict(eval_model(baseline, dataloaders_dict_56['val']), orient='index')

3it [00:44, 14.84s/it]


,0
accuracy,0.750000
balanced_accuracy,0.750000
auc,0.822779
f1,0.767677
precision,0.716981
tpr,0.826087
fpr,0.326087
tnr,0.673913
fnr,0.173913


In [ ]:
pd.DataFrame.from_dict(eval_model(model_full, dataloaders_dict_combined['val']), orient='index')

12it [01:48,  9.05s/it]


,0
accuracy,0.833795
balanced_accuracy,0.818310
auc,0.908404
f1,0.864865
precision,0.824034
tpr,0.909953
fpr,0.273333
tnr,0.726667
fnr,0.090047


In [ ]:
pd.DataFrame.from_dict(eval_model(model_full, dataloaders_dict_12['val']), orient='index')

9it [01:36, 10.69s/it]


,0
accuracy,0.825279
balanced_accuracy,0.795367
auc,0.902972
f1,0.866856
precision,0.813830
tpr,0.927273
fpr,0.336538
tnr,0.663462
fnr,0.072727


In [ ]:
pd.DataFrame.from_dict(eval_model(model_full, dataloaders_dict_56['val']), orient='index')

3it [00:51, 17.11s/it]


,0
accuracy,0.858696
balanced_accuracy,0.858696
auc,0.916824
f1,0.857143
precision,0.866667
tpr,0.847826
fpr,0.130435
tnr,0.869565
fnr,0.152174


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_vanilla, dataloaders_dict_combined['val']), orient='index')

12it [03:40, 18.35s/it]


,0
accuracy,0.853186
balanced_accuracy,0.847425
auc,0.884645
f1,0.875294
precision,0.869159
tpr,0.881517
fpr,0.186667
tnr,0.813333
fnr,0.118483


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_vanilla, dataloaders_dict_12['val']), orient='index')

9it [02:20, 15.57s/it]


,0
accuracy,0.862454
balanced_accuracy,0.852331
auc,0.904545
f1,0.888889
precision,0.880952
tpr,0.896970
fpr,0.192308
tnr,0.807692
fnr,0.103030


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_vanilla, dataloaders_dict_56['val']), orient='index')

0it [00:00, ?it/s]

In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_label, dataloaders_dict_combined['val']), orient='index')

12it [01:18,  6.52s/it]


,0
accuracy,0.855956
balanced_accuracy,0.853649
auc,0.921327
f1,0.875598
precision,0.884058
tpr,0.867299
fpr,0.160000
tnr,0.840000
fnr,0.132701


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_label, dataloaders_dict_12['val']), orient='index')

9it [00:59,  6.65s/it]


,0
accuracy,0.862454
balanced_accuracy,0.857663
auc,0.931935
f1,0.886850
precision,0.895062
tpr,0.878788
fpr,0.163462
tnr,0.836538
fnr,0.121212


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_label, dataloaders_dict_56['val']), orient='index')

3it [00:27,  9.02s/it]


,0
accuracy,0.836957
balanced_accuracy,0.836957
auc,0.879017
f1,0.835165
precision,0.844444
tpr,0.826087
fpr,0.152174
tnr,0.847826
fnr,0.173913


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_sens, dataloaders_dict_combined['val']), orient='index')

12it [01:11,  5.92s/it]


,0
accuracy,0.850416
balanced_accuracy,0.847946
auc,0.926161
f1,0.870813
precision,0.879227
tpr,0.862559
fpr,0.166667
tnr,0.833333
fnr,0.137441


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_sens, dataloaders_dict_12['val']), orient='index')

9it [00:52,  5.89s/it]


,0
accuracy,0.851301
balanced_accuracy,0.845017
auc,0.932925
f1,0.878049
precision,0.883436
tpr,0.872727
fpr,0.182692
tnr,0.817308
fnr,0.127273


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_sens, dataloaders_dict_56['val']), orient='index')

3it [00:18,  6.05s/it]


,0
accuracy,0.847826
balanced_accuracy,0.847826
auc,0.897921
f1,0.844444
precision,0.863636
tpr,0.826087
fpr,0.130435
tnr,0.869565
fnr,0.173913


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_joint, dataloaders_dict_combined['val']), orient='index')

12it [01:14,  6.21s/it]


,0
accuracy,0.853186
balanced_accuracy,0.853207
auc,0.923539
f1,0.871671
precision,0.891089
tpr,0.853081
fpr,0.146667
tnr,0.853333
fnr,0.146919


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_joint, dataloaders_dict_12['val']), orient='index')

9it [00:53,  5.94s/it]


,0
accuracy,0.851301
balanced_accuracy,0.848572
auc,0.926981
f1,0.876543
precision,0.893082
tpr,0.860606
fpr,0.163462
tnr,0.836538
fnr,0.139394


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_joint, dataloaders_dict_56['val']), orient='index')

3it [00:20,  6.92s/it]


,0
accuracy,0.858696
balanced_accuracy,0.858696
auc,0.910208
f1,0.853933
precision,0.883721
tpr,0.826087
fpr,0.108696
tnr,0.891304
fnr,0.173913


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_kamiran, dataloaders_dict_combined['val']), orient='index')

12it [01:18,  6.58s/it]


,0
accuracy,0.850416
balanced_accuracy,0.845055
auc,0.914502
f1,0.872642
precision,0.868545
tpr,0.876777
fpr,0.186667
tnr,0.813333
fnr,0.123223


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_kamiran, dataloaders_dict_12['val']), orient='index')

9it [00:59,  6.63s/it]


,0
accuracy,0.858736
balanced_accuracy,0.849301
auc,0.914277
f1,0.885542
precision,0.880240
tpr,0.890909
fpr,0.192308
tnr,0.807692
fnr,0.109091


In [ ]:
pd.DataFrame.from_dict(eval_model(ensemble_kamiran, dataloaders_dict_56['val']), orient='index')

3it [00:25,  8.45s/it]


,0
accuracy,0.826087
balanced_accuracy,0.826087
auc,0.917297
f1,0.826087
precision,0.826087
tpr,0.826087
fpr,0.173913
tnr,0.826087
fnr,0.173913


In [ ]:
pd.DataFrame.from_dict(eval_model(gdro, dataloaders_dict_combined['val']), orient='index')

12it [00:41,  3.45s/it]


,0
accuracy,0.637119
balanced_accuracy,0.629826
auc,0.680221
f1,0.684337
precision,0.696078
tpr,0.672986
fpr,0.413333
tnr,0.586667
fnr,0.327014


In [ ]:
pd.DataFrame.from_dict(eval_model(gdro, dataloaders_dict_12['val']), orient='index')

9it [00:34,  3.81s/it]


,0
accuracy,0.639405
balanced_accuracy,0.626078
auc,0.694056
f1,0.699690
precision,0.715190
tpr,0.684848
fpr,0.432692
tnr,0.567308
fnr,0.315152


In [ ]:
pd.DataFrame.from_dict(eval_model(gdro, dataloaders_dict_56['val']), orient='index')

3it [00:16,  5.39s/it]


,0
accuracy,0.630435
balanced_accuracy,0.630435
auc,0.623346
f1,0.630435
precision,0.630435
tpr,0.630435
fpr,0.369565
tnr,0.630435
fnr,0.369565


In [ ]:
pd.DataFrame.from_dict(eval_model(coral, dataloaders_dict_combined['val']), orient='index')

12it [00:37,  3.08s/it]


,0
accuracy,0.631579
balanced_accuracy,0.620269
auc,0.684866
f1,0.685579
precision,0.683962
tpr,0.687204
fpr,0.446667
tnr,0.553333
fnr,0.312796


In [ ]:
pd.DataFrame.from_dict(eval_model(coral, dataloaders_dict_12['val']), orient='index')

9it [00:27,  3.03s/it]


,0
accuracy,0.657993
balanced_accuracy,0.644784
auc,0.706935
f1,0.716049
precision,0.729560
tpr,0.703030
fpr,0.413462
tnr,0.586538
fnr,0.296970


In [ ]:
pd.DataFrame.from_dict(eval_model(coral, dataloaders_dict_56['val']), orient='index')

3it [00:10,  3.38s/it]


,0
accuracy,0.554348
balanced_accuracy,0.554348
auc,0.612949
f1,0.585859
precision,0.547170
tpr,0.630435
fpr,0.521739
tnr,0.478261
fnr,0.369565


In [ ]:
pd.DataFrame.from_dict(eval_model(cdan, dataloaders_dict_combined['val']), orient='index')

12it [00:37,  3.15s/it]


,0
accuracy,0.648199
balanced_accuracy,0.642196
auc,0.692133
f1,0.692494
precision,0.707921
tpr,0.677725
fpr,0.393333
tnr,0.606667
fnr,0.322275


In [ ]:
pd.DataFrame.from_dict(eval_model(cdan, dataloaders_dict_12['val']), orient='index')

9it [00:27,  3.10s/it]


,0
accuracy,0.650558
balanced_accuracy,0.636946
auc,0.682867
f1,0.709877
precision,0.723270
tpr,0.696970
fpr,0.423077
tnr,0.576923
fnr,0.303030


In [ ]:
pd.DataFrame.from_dict(eval_model(cdan, dataloaders_dict_56['val']), orient='index')

3it [00:10,  3.40s/it]


,0
accuracy,0.641304
balanced_accuracy,0.641304
auc,0.708412
f1,0.629213
precision,0.651163
tpr,0.608696
fpr,0.326087
tnr,0.673913
fnr,0.391304


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_label, dataloaders_dict_combined['val']), orient='index')

12it [00:35,  2.99s/it]


,0
accuracy,0.792244
balanced_accuracy,0.785656
auc,0.844550
f1,0.822695
precision,0.820755
tpr,0.824645
fpr,0.253333
tnr,0.746667
fnr,0.175355


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_label, dataloaders_dict_12['val']), orient='index')

9it [00:32,  3.65s/it]


,0
accuracy,0.799257
balanced_accuracy,0.783042
auc,0.839394
f1,0.839286
precision,0.824561
tpr,0.854545
fpr,0.288462
tnr,0.711538
fnr,0.145455


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_label, dataloaders_dict_56['val']), orient='index')

3it [00:15,  5.31s/it]


,0
accuracy,0.771739
balanced_accuracy,0.771739
auc,0.836011
f1,0.758621
precision,0.804878
tpr,0.717391
fpr,0.173913
tnr,0.826087
fnr,0.282609


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_sens, dataloaders_dict_combined['val']), orient='index')

12it [00:37,  3.10s/it]


,0
accuracy,0.795014
balanced_accuracy,0.780316
auc,0.870742
f1,0.831818
precision,0.799127
tpr,0.867299
fpr,0.306667
tnr,0.693333
fnr,0.132701


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_sens, dataloaders_dict_12['val']), orient='index')

9it [00:26,  2.96s/it]


,0
accuracy,0.788104
balanced_accuracy,0.761509
auc,0.862354
f1,0.835735
precision,0.796703
tpr,0.878788
fpr,0.355769
tnr,0.644231
fnr,0.121212


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_sens, dataloaders_dict_56['val']), orient='index')

3it [00:09,  3.29s/it]


,0
accuracy,0.815217
balanced_accuracy,0.815217
auc,0.879017
f1,0.817204
precision,0.808511
tpr,0.826087
fpr,0.195652
tnr,0.804348
fnr,0.173913


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_joint, dataloaders_dict_combined['val']), orient='index')

12it [01:35,  7.99s/it]


,0
accuracy,0.786704
balanced_accuracy,0.773207
auc,0.874123
f1,0.823799
precision,0.796460
tpr,0.853081
fpr,0.306667
tnr,0.693333
fnr,0.146919


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_joint, dataloaders_dict_12['val']), orient='index')

9it [01:11,  7.90s/it]


,0
accuracy,0.791822
balanced_accuracy,0.766317
auc,0.874942
f1,0.838150
precision,0.801105
tpr,0.878788
fpr,0.346154
tnr,0.653846
fnr,0.121212


In [ ]:
pd.DataFrame.from_dict(eval_model(full_layer_joint, dataloaders_dict_56['val']), orient='index')

3it [00:25,  8.42s/it]


,0
accuracy,0.771739
balanced_accuracy,0.771739
auc,0.856333
f1,0.769231
precision,0.777778
tpr,0.760870
fpr,0.217391
tnr,0.782609
fnr,0.239130
